In [ ]:
'''importing packages'''


import time
import numpy
import pickle
import nltk
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string


'''TIME'''
start_time=time.time()


stop_words = set(stopwords.words('english'))
porter = PorterStemmer()
import pandas as pd



'''CHANGE THE PATH TO THE PATH OF THE TRAIN DATA (FORM: LABEL..REVIEW)'''


X=pd.read_csv("C:/Users/DEEPANSH/Desktop/as2.csv",names = ["label", "review"])
t=[',','.','/',',','!','?']


'''PREPROCESSING REMOVAL OF STOPWORDS AND PUNCTUATIONS + STEMMING'''
def clean_str(string):
    w = nltk.word_tokenize(string)
    w = [p.lower() for p in w if not p in stop_words]
    s = [p.translate(t) for p in w]
    w = [porter.stem(word) for word in w]
    string = " ".join(w)
    return string.strip().lower()


'''SUMMARISING DATA AND FORMING VOCAB'''
list_of_all_lines=list(X['review'])
#list_of_all_lines = [clean_str(str(sent)) for sent in list_of_all_lines]
m=len(list_of_all_lines)
#Initialising a vocablury in a set
vocablury = set()


for i in list_of_all_lines:
	#Extracting every word possible in the file
	#lower() converts every word in small so that 
	#'This' and 'this' are treated as the same word
	tmp = str(i).lower().split()
	vocablury.update(tmp[0:])
#Converting into list so that pickle can write it as list easily
Matrix_X=[]
Matrix_Y=[]
vocablury=list(vocablury)
y_labels=[1,2,3,4,5]
y_labels=list(y_labels)
for i in X['label']:
    y_labelled=i
    dictionary_for_label_of_a_line=dict.fromkeys(y_labels,0)
    dictionary_for_label_of_a_line[y_labelled]=1
    row_y=list(dictionary_for_label_of_a_line.values())
    Matrix_Y.append(row_y)


#Converting the data into appropriate input and keeping 
#number of count every word has in the article 
#and then extracting the values of the counter dictionary intov
#another list and then putting this list in another list which
#will ultimately form a list of list which will be the resultant matrix

for inp in list_of_all_lines:
	tmp=str(inp).lower().split()
	x_inputs=tmp[1:]

	# Making a dictionary for every line  in which every  
	# element of dictionary is mapped to 0
	dictionary_for_every_line=dict.fromkeys(vocablury,0)

	for j in x_inputs:
		# Counting the Ocuurance of the word j in the line  
		dictionary_for_every_line[j]=dictionary_for_every_line[j]+1
	#Extracting the list of values in the counter dictionary
	row_x=list(dictionary_for_every_line.values())
	Matrix_X.append(row_x)

	#Making a similar dictionary for y_labels in which the label which 
	#is tagged will be given a value 1
    

#Sum of column of the Matrix_Y
Matrix_Y=numpy.array(Matrix_Y)
y_label_column_sum=numpy.sum(Matrix_Y,axis=0)
y_label_column_sum=y_label_column_sum.astype(float)

'''MAIN ALGO'''
#Calculating Phi(i)
Phi_List=numpy.divide(y_label_column_sum,m)

#Calculating Theta_Matrix
Matrix_X=numpy.array(Matrix_X).astype(float)
Theta_Matrix=Matrix_X.transpose().dot(Matrix_Y)


'''LAPLACE SMOOTHING C=1'''
#Adding c=1 to every element 
# Laplace Smoothing 
Theta_Matrix=numpy.add(Theta_Matrix,1)

#Sum of elements in every column of X_transpose_Y
X_transpose_Y_column_sum=numpy.sum(Theta_Matrix,axis=0)

Theta_Matrix=numpy.divide(Theta_Matrix,X_transpose_Y_column_sum)
#.npy format is platform independent and in binary format 
#not understandable by us and more efficient

numpy.save('Theta_Matrix.npy',Theta_Matrix)

numpy.save('Phi_List.npy',Phi_List)


'''TEMPORARY FILE FOR VOCAB AND LABELS'''
#Vocablury file save in a temporary file
with open('vocablury.txt','wb') as file_handler:
	pickle.dump(vocablury,file_handler)
#y_labels file save in a temporary file
with open('y_labels.txt','wb') as file_handler:
	pickle.dump(y_labels,file_handler)

'''DISPLAYING TOTAL TIME HENCE SHOWING COMPLETION'''
end_time=time.time()
print("Time Elapsed = " + str(end_time-start_time))


# We have to save the vocablury set in a file because we are not allowed to
# update the vocablury according to the test data and any new word if 
# came then we ignore that word as if never existed


#-----------------------------------------------------------#

In [ ]:

'''IMPORTING PACKAGES'''
import time
import numpy
import pickle
import nltk
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string
start_time=time.time()
#Opening a File providing only the read access
stop_words = set(stopwords.words('english'))
porter = PorterStemmer()
import pandas as pd



'''CHANGE THE PATH TO THE TRAINING DATA (FORM: LABEL:REVIEW)'''
X=pd.read_csv("C:/Users/DEEPANSH/Desktop/as2t.csv",names = ["label", "review"])



'''PREPROCESSING OF DATA REMOVAL OF STOPWORDS AND PUNCTUATIONS + STEMMING'''
t=[',','.','/',',','!','?']
def clean_str(string):
    w = nltk.word_tokenize(string)
    w = [p.lower() for p in w]
    s = [p.translate(t) for p in w]
    w = [word for word in s if word.isalpha()]
    w = [p for p in w if not p in stop_words]
    w = [porter.stem(word) for word in w]
    string = " ".join(w)
    return string.strip().lower()



'''SUMMARISING DATA'''
list_of_all_lines=list(X['review'])
#list_of_all_lines = [clean_str(str(sent)) for sent in list_of_all_lines]
m=len(list_of_all_lines)


'''LOADING TEMPORARY FILE'''
with open('vocablury.txt','rb') as file_handler:
	vocablury=pickle.load(file_handler)

    
    
Phi_List=numpy.load('Phi_List.npy')

Theta_Matrix=numpy.load('Theta_Matrix.npy')

# Converting the data into appropriate input and keeping 
# number of count every word has in the article 
# and then extracting the values of the counter dictionary intov
# another list and then putting this list in another list which
# will ultimately form a list of list which will be the resultant matrix
Matrix_X=[]
Matrix_Y=[]
for i in list_of_all_lines:
	tmp=i.lower().split()
	x_inputs=tmp[0:]

	# Making a dictionary in which every element is mapped to 0
	dictionary_for_every_line=dict.fromkeys(vocablury,0)

	for j in x_inputs:
		if(j in vocablury):
			dictionary_for_every_line[j]=dictionary_for_every_line[j]+1
	# Extracting the list of values in the counter dictionary
	row_x=list(dictionary_for_every_line.values())
	Matrix_X.append(row_x)


y_labels=[1,2,3,4,5]
y_labels=list(y_labels)
for i in X['label']:
    y_labelled=i
    dictionary_for_label_of_a_line=dict.fromkeys(y_labels,0)
    dictionary_for_label_of_a_line[y_labelled]=1
    row_y=list(dictionary_for_label_of_a_line.values())
    Matrix_Y.append(row_y)


Matrix_X=numpy.array(Matrix_X)
Matrix_Y=numpy.array(Matrix_Y)

Matrix_Y_into_Column_Y_can_be_called_as_original=[]
# Putting observed y_labels into a column 
for i in Matrix_Y:
	count=0
	for j in i:
		count=count+1
		if(j==1):
			Matrix_Y_into_Column_Y_can_be_called_as_original.append(count)
			break

            
            
            
'''STORING ORIGNAL OR EXPECTED OUTPUTS IN A TEMPORARY FILE'''
with open('Matrix_Y_into_Column_Y_can_be_called_as_original.txt','wb') as file_handler:
	pickle.dump(Matrix_Y_into_Column_Y_can_be_called_as_original,file_handler)

    
    
'''Hessian Matrix''' 
H_Matrix = numpy.log(Phi_List)+(Matrix_X.dot(numpy.log(Theta_Matrix)))

# The Given Data
Y_labels_extracted_from_test_data=[]
# Here we have classes 1,2,3,4,5,6,7,8 and so here count variable 
# will also be the class predicted 
for i in H_Matrix:
	max_element=numpy.amax(i)
	count=0
	for j in i:
		count=count+1
		if(j==max_element):
			Y_labels_extracted_from_test_data.append(count)
			break

            
            
'''STORING PREDICTED CLASS BY THE MODEL TEMPORARILY'''
with open('Y_labels_extracted_from_test_data.txt','wb') as file_handler:
	pickle.dump(Y_labels_extracted_from_test_data,file_handler)

    
'''DISPLAYING TOTAL TIME HENCE SHOWING COMPLETION'''
end_time = time.time()
print("Total Time Elapsed " +str(end_time-start_time))



In [ ]:

'''IMPORTING PACKAGES'''
import pickle
import time


'''TIME FOR ACCOUNT'''
start_time=time.time()
# Observed Data/Classification


'''TEMPORARY FILES CALLED TO GET THE PREDICTED AND ACTUAL VALUE'''
with open('Matrix_Y_into_Column_Y_can_be_called_as_original.txt','rb') as file_handler:
	Matrix_Y_into_Column_Y_can_be_called_as_original=pickle.load(file_handler)
# Predicted Data/Classification
with open('Y_labels_extracted_from_test_data.txt','rb') as file_handler:
	Y_labels_extracted_from_test_data=pickle.load(file_handler)

m1=len(Matrix_Y_into_Column_Y_can_be_called_as_original)
m2=len(Y_labels_extracted_from_test_data)
# Since m1 and m2 are same we can run loop on any one of them
count=0.0

'''CLACULATING ACCURACY BY THE CLASSICAL PROBLEMS CORRECT/TOTAL'''
for i in range(m1):
	if(Matrix_Y_into_Column_Y_can_be_called_as_original[i]==Y_labels_extracted_from_test_data[i]):
		count=count+1
accuracy=count/m1*100
print("The Accuracy of the above Model is "+str(accuracy))

'''DISPLAYING TOTAL TIME HENCE SHOWING COMPLETION'''
end_time=time.time()
print("Time Elapsed "+str(end_time-start_time))